In [230]:
import numpy as np
import itertools 
import operator
from datetime import datetime
import sys

# password possible char
# a-z    A-Z     1234567890      @%+\/'!#$^?:,.(){}[]~-_*   26+26+10+24

In [ ]:
# credit from https://songhuiming.github.io/pages/2017/08/20/build-recurrent-neural-network-from-scratch/
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)


'''
 - model: 
 - X_train:
 - y_train:
 - learning_rate:
 - nepoch:
 - evaluate loss_after:
'''
def train_with_sgd(model, X_train, y_train, learning_rate = 0.005, nepoch = 100, evaluate_loss_after = 5):
    # keep track of the losses so that we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("%s: loss after num_examples_seen=%d epoch=%d: %f" %(time, num_examples_seen, epoch, loss))
            # adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5
                print("setting learning rate to %f" %(learning_rate))
            sys.stdout.flush()
        # for each training example...
        for i in range(len(y_train)):
            # one sgd step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1


In [246]:
class RNNNumpy():
    def __init__(self, word_dim, hidden_dim = 100, bptt_truncate = 4):
        # assign instance variable
        self.word_dim = word_dim   # number of possible characters, in this case 86
        self.hidden_dim = hidden_dim # number of hidden units 
        self.bptt_truncate = bptt_truncate
        
        # random initiate the parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
    
    def forward_progagation(self, x):
        # total time steps / length of the password
        T = len(x)

        #intialize s, with the initial state s set to zero, we thus have T+1 rows, each with size of hidden_dim
        s = np.zeros((T+1, self.hidden_dim))
        #intialize o, 
        o = np.zeros((T, self.word_dim))

        for t in np.arange(T):
            # since x is a one-hot encoder, index U with x[t] is the same thing as mutiply U with tons of 0s
            s[t] = np.tanh(self.U[:, x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t])) 
        return [o, s]

    def predict(self, x):
        o, s = self.forward_progagation(x)
        return np.argmax(o, axis = 1)
    
    
    # in this case x and y are 2-d arrays
    def calculate_total_loss(self, x, y):
        L = 0
        # for each sentence ...
        for i in np.arange(len(y)):
            o, s = self.forward_progagation(x[i])
            # we only care about our prediction of the "correct" words
            correct_word_predictions = o[np.arange(len(y[i])), y[i]]
            # add to the loss based on how off we were
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L

    # in this case x and y are 2-d arrays
    def calculate_loss(self, x, y):
        # divide the total loss by the number of training examples
        N = np.sum((len(y_i) for y_i in y))
        return self.calculate_total_loss(x, y)/N
    
    
    def bptt(self, x, y):
        T = len(y)
        # perform forward propagation
        o, s = self.forward_progagation(x)
        # we will accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1   # it is y_hat - y
        # for each output backwards ...
        for t in np.arange(T):
            dLdV += np.outer(delta_o[t], s[t].T)    # at time step t, shape is word_dim * hidden_dim
            # initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
            # backpropagation through time (for at most self.bptt_truncate steps)
            # given time step t, go back from time step t, to t-1, t-2, ...
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print("Backprogation step t=%d bptt step=%d" %(t, bptt_step))
                dLdW += np.outer(delta_t, s[bptt_step - 1])
                dLdU[:, x[bptt_step]] += delta_t
                # update delta for next step
                dleta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1]**2)
        return [dLdU, dLdV, dLdW]
    
            
    def sgd_step(self, x, y, learning_rate):
        dLdU, dLdV, dLdW = self.bptt(x, y)
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW



In [247]:
# https://theasciicode.com.ar/ascii-control-characters/backspace-ascii-code-8.html
# set range for char 32 - 126 in ascii,  95 chars, remove any pw cotains char outside the range
# also we want an addtional char END indicates the end of the password, so totally 96 chars
# END would only appear in y 
s =  "aab!@ 12bbcc"
# y = ab!@ 12bbccEND where END indicate it ends
l = []
for i in s:
    l.append(ord(i)-32)
print(l)

def generate_y(l):
    y = l[1:]
    y.append(95)
    return y

print(generate_y(l))

[65, 65, 66, 1, 32, 0, 17, 18, 66, 66, 67, 67]
[65, 66, 1, 32, 0, 17, 18, 66, 66, 67, 67, 95]


In [248]:
np.random.seed(10)
char_size = 96 
model = RNNNumpy(char_size)
o, s = model.forward_progagation(l)

print(o.shape)
print(o)

predictions = model.predict(l)
print(predictions.shape)
print(predictions) 


(12, 96)
[[0.01033776 0.0102209  0.0096735  ... 0.01123267 0.01032486 0.01071546]
 [0.01022094 0.01038388 0.00980454 ... 0.01099515 0.01026953 0.01064471]
 [0.01047122 0.01105342 0.01098961 ... 0.01004628 0.01021592 0.01071909]
 ...
 [0.01050321 0.01108562 0.01087564 ... 0.01055242 0.0102316  0.01057934]
 [0.01028706 0.01126629 0.01036227 ... 0.01056727 0.01106294 0.01014812]
 [0.01043169 0.01113691 0.01024768 ... 0.0104929  0.01119632 0.01027299]]
(12,)
[29 12 43 78 71 35 95 33 44 44 57 57]


In [254]:
f = open("test.txt", "r")
line = f.readline()
x = []
y = []
while line != "":
    l = [ord(i)-32 for i in line][:-1] # convert to int
    x.append(l) # add to X
    print(l)
    l.append(95)
    l = l[1:]
    y.append(l) # modify and add to y
    print(l)
    line = f.readline()

   

[82, 79, 67, 75, 89, 79, 85]
[79, 67, 75, 89, 79, 85, 95]
[17, 18, 19, 20, 21, 22, 23, 24]
[18, 19, 20, 21, 22, 23, 24, 95]
[65, 66, 67, 17, 18, 19]
[66, 67, 17, 18, 19, 95]
[78, 73, 67, 79, 76, 69]
[73, 67, 79, 76, 69, 95]
[68, 65, 78, 73, 69, 76]
[65, 78, 73, 69, 76, 95]
[66, 65, 66, 89, 71, 73, 82, 76]
[65, 66, 89, 71, 73, 82, 76, 95]
[77, 79, 78, 75, 69, 89]
[79, 78, 75, 69, 89, 95]
[74, 79, 78, 78, 65, 73, 83, 69, 17]
[79, 78, 78, 65, 73, 83, 69, 17, 95]
[68, 74, 79, 78, 78, 65, 72]
[74, 79, 78, 78, 65, 72, 95]
[16, 24, 23, 19, 21, 25]
[24, 23, 19, 21, 25, 95]
[16, 24, 23, 19, 21, 24, 25, 24, 24, 20]
[24, 23, 19, 21, 24, 25, 24, 24, 20, 95]
[16, 24, 23, 19, 21, 24, 25, 22, 20, 21]
[24, 23, 19, 21, 24, 25, 22, 20, 21, 95]
[16, 19, 17, 18, 65, 90, 83, 68]
[19, 17, 18, 65, 90, 83, 68, 95]
[16, 19, 17, 18, 65, 80]
[19, 17, 18, 65, 80, 95]
[16, 19, 17, 18, 65, 76, 76, 73, 83, 84, 69, 82]
[19, 17, 18, 65, 76, 76, 73, 83, 84, 69, 82, 95]
[16, 19, 17, 18, 24, 25, 13, 80, 73, 77, 80, 78, 7

In [250]:
print("Expected Loss for random prediction: %f" % np.log(char_size))
print("Actual loss: %f" % model.calculate_loss(x, y))

Expected Loss for random prediction: 4.564348
Actual loss: 4.566333


/Users/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:88: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


In [251]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate = 1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

performing gradient check for parameter U with size 1000. 
Gradient check error: parameter = U ix = (0, 3)
+h Loss: 18.307080
-h Loss: 18.307296
Estimated gradient: -0.108091
Backpropagation gradient: -0.108091
Relative error: 0.000000


In [253]:
np.random.seed(10)
model = RNNNumpy(96)
losses = train_with_sgd(model, x, y, nepoch = 10, evaluate_loss_after = 1)

print("Actual loss: %f" % model.calculate_loss(x, y))

2020-05-16 17:54:15: loss after num_examples_seen=0 epoch=0: 4.566333
2020-05-16 17:54:15: loss after num_examples_seen=33 epoch=1: 4.547854


/Users/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:88: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


2020-05-16 17:54:15: loss after num_examples_seen=66 epoch=2: 4.528269
2020-05-16 17:54:15: loss after num_examples_seen=99 epoch=3: 4.505158
2020-05-16 17:54:15: loss after num_examples_seen=132 epoch=4: 4.473565
2020-05-16 17:54:15: loss after num_examples_seen=165 epoch=5: 4.416195
2020-05-16 17:54:15: loss after num_examples_seen=198 epoch=6: 4.187101
2020-05-16 17:54:15: loss after num_examples_seen=231 epoch=7: 3.919399
2020-05-16 17:54:15: loss after num_examples_seen=264 epoch=8: 3.757771
2020-05-16 17:54:15: loss after num_examples_seen=297 epoch=9: 3.631927
Actual loss: 3.537425


In [ ]:
np.random.seed(10)
char_size = 96 
model = RNNNumpy(char_size)

# 1. Insert previous U, V, W into model 

# 2. Open rockyou.txt, get string and start training
losses = train_with_sgd(model, x, y, nepoch = 10, evaluate_loss_after = 1)
# training end 

# 3. Generate random passwords, then check for 

In [ ]:

# possibily not going to use this  
    def gradient_check(self, x, y, h = 0.001, error_threshold = 0.01):
        # calculate the gradient using backpropagation
        bptt_gradients = self.bptt(x, y)
        # list of all params we want to check
        model_parameters = ["U", "V", "W"]
        # gradient check for each parameter
        for pidx, pname in enumerate(model_parameters):
            # get the actual parameter value from model, e.g. model.W
            parameter = operator.attrgetter(pname)(self)
            print("performing gradient check for parameter %s with size %d. " %(pname, np.prod(parameter.shape)))
            # iterate over each element of the parameter matrix, e.g. (0,0), (0,1)...
            it = np.nditer(parameter, flags = ['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # save the original value so we can reset it later
                original_value = parameter[ix]
                # estimate the gradient using (f(x+h) - f(x-h))/2h
                parameter[ix] = original_value + h
                gradplus = self.calculate_total_loss([x], [y])
                parameter[ix] = original_value - h
                gradminus = self.calculate_total_loss([x], [y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                # reset parameter to the original value
                parameter[ix] = original_value
                # the gradient for this parameter calculated using backpropagation
                backprop_gradient = bptt_gradients[pidx][ix]
                # calculate the relative error (|x - y|)/(|x|+|y|)
                relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
                # if the error is too large fail the gradient check
                if relative_error < error_threshold:
                    print("Gradient check error: parameter = %s ix = %s" %(pname, ix))
                    print("+h Loss: %f" % gradplus)
                    print("-h Loss: %f" % gradminus)
                    print("Estimated gradient: %f" % estimated_gradient)
                    print("Backpropagation gradient: %f" % backprop_gradient)
                    print("Relative error: %f" % relative_error)
                    return
                it.iternext()
            print("Gradient check for parameter %s passed. " %(pname))